### Data Collection II
Making it easier to access multiple series of the M3 survey from Census Bureau

In [1]:
import pandas as pd
import requests
import json
import os 
from bs4 import BeautifulSoup as bs 
import re
from collections import OrderedDict as od
from pypdf import PdfReader 

In [ ]:
os.chdir("C:/Users/marvi/OneDrive/Documents/GitHub/Scraping-and-APIs")
os.getcwd()

In [45]:
def wrangling(item):
    res = [idx for idx, chr in enumerate(item) if chr.isupper()]    
    print(res)
    #return(item)

In [46]:
string = "MarkingMyTerritory"
#re.findall("([A-Z])")
wrangling(string)

[0, 7, 9]


#### Census Bureau
[M3 categories](https://www2.census.gov/programs-surveys/m3/technical-documentation/code-lists/sichist.pdf)

In [2]:
census = requests.get("https://api.census.gov/data/timeseries/eits/m3.json?get=cell_value,data_type_code,time_slot_id,seasonally_adj,category_code,error_data&for=US&time=from+1992-01+to+2024-03&key=a9c6bd12867708842174babc90f20cfaa20255a7")
census = census.json()
cbs = pd.DataFrame(census, columns = census[0])
cbs = cbs[cbs["data_type_code"] == 'VS'].reset_index(drop = True)

KeyboardInterrupt: 

In [ ]:
#Grabs and sorts unique values for the category code
b = cbs["category_code"].tolist()
cat_abv = list(od.fromkeys(b))
cat_abv = sorted(cat_abv)

In [2]:
reader = PdfReader('m3_codes.pdf')
print(len(reader.pages))
text = ""

for i in range(0, len(reader.pages)):
    text += reader.pages[i].extract_text()
text = str(text)

o_text = text

separator = [" ", ";", ",", "\'",".",":"]
for sep in separator:
    text = text.replace(sep, "")

text = text[105:]
text

10


In [43]:
#makes a list on a supposed pattern
clean = re.findall("[A-Z|0-90-9A-Z]{3}.*\W", text)[2:-8]

#first iterate through each string object
for item in clean: 
    if(item != clean[-9]):
        #makes the list of codes 
        list_2 = re.findall("[A-Z|0-90-9A-Z]{3}", item)
        #finds index start point for each code
        start = [re.search(stuff, item).start() for stuff in list_2]
        #creating proxy list for items with 1+ code and industry
        x_2 = [] 
        #this loop looks through string for code and specific industry or cat
        for i in range(0, len(start)):
            
            if start[i] == start[-1]:
                l_item = item[start[i]:-1]
            else:
                l_item = item[start[i]: start[i+1]]

            #appends the proxy list to main list
            x_2.append(l_item)
        
        clean_2.extend(x_2)
    else:
        break

clean_2

['MTMTotalManufacturing',
 'MXTManufacturingExcludingTransportation',
 'MTUAllManufacturingwithUnfilledOrders',
 'MXDManufacturingExcludingDefense',
 'DXDDurableExcludingDefense',
 'MDMDurableGoodsTotal',
 'MDUDurableGoodswithUnfilledOrders',
 'MNMNondurableGoodsTotal',
 'MNUNondurableGoodswithUnfilledOrders',
 'AUEAutomotiveEquipment',
 'HGAHomeGoodsandApparel',
 'COSConsumerStaples',
 'MAEMachineryandEquipment',
 'BUSBusinessSupplies',
 'CMSConstructionSupplies',
 'DEPDefenseProducts',
 'OMPOtherMaterialsSuppliesandIntermediateProducts',
 'TCGTotalCapitalGoods',
 'NDENondefenseCapitalGoods',
 'NXANondefenseCapitalGoodsExcludingAircraftandParts',
 'DEFDefenseCapitalGoods',
 'DXCDurablesExcludingCapitalGoods',
 'DXADurablesExcludingAircraftandParts',
 'N4XNondefenseAircraftMissilesSpaceVehiclesandParts',
 'N5XNondefenseShipbuildingTankComponentsand',
 'D4XDefenseAircraftMissilesSpaceVehiclesandParts',
 'D5XDefenseShipbuildingTanksandOrdnance',
 'SX4AircraftMissilesSpaceVehiclesandParts

In [44]:
cgy_dictionary = {"category_code": [], "category_name": []}

for item in clean_2:
    cgy_dictionary["category_code"].append(item[0:3])
    cgy_dictionary["category_name"].append(wrangling(item[3:]))

pd.DataFrame(cgy_dictionary)

,category_code,category_name
0,MTM,TotalManufacturing
1,MXT,ManufacturingExcludingTransportation
2,MTU,AllManufacturingwithUnfilledOrders
3,MXD,ManufacturingExcludingDefense
4,DXD,DurableExcludingDefense
...,...,...
298,NEC,
299,30C,MiscellaneousPlasticsProducts
300,NEC,
301,31A,LeatherTanningandFinishingandFootwearCutStock


In [24]:
#run if statement that checks each string in the category name column

for 

['MTMTotalManufacturing\n',
 'MXTManufacturingExcludingTransportation\n',
 'MTUAllManufacturingwithUnfilledOrders\n',
 'MXDManufacturingExcludingDefense\n',
 'DXDDurableExcludingDefense\n',
 'MDMDurableGoodsTotalMDUDurableGoodswithUnfilledOrders\n',
 'MNMNondurableGoodsTotal\n',
 'MNUNondurableGoodswithUnfilledOrdersAUEAutomotiveEquipment\n',
 'HGAHomeGoodsandApparelCOSConsumerStaples\n',
 'MAEMachineryandEquipment\n',
 'BUSBusinessSupplies\n',
 'CMSConstructionSuppliesDEPDefenseProducts\n',
 'OMPOtherMaterialsSuppliesandIntermediateProducts\n',
 'TCGTotalCapitalGoodsNDENondefenseCapitalGoods\n',
 'NXANondefenseCapitalGoodsExcludingAircraftandParts\n',
 'DEFDefenseCapitalGoodsDXCDurablesExcludingCapitalGoodsDXADurablesExcludingAircraftandParts\n',
 'N4XNondefenseAircraftMissilesSpaceVehiclesandParts\n',
 'N5XNondefenseShipbuildingTankComponentsand\n',
 'D4XDefenseAircraftMissilesSpaceVehiclesandParts\n',
 'D5XDefenseShipbuildingTanksandOrdnance\n',
 'SX4AircraftMissilesSpaceVehiclesand

In [16]:
f = [0]
f[0]

0